In [28]:
import pandas as pd
import numpy as np

In [29]:
df_pbp = pd.read_csv(r"RAW/nhl_pbp_20212022.csv")
df_shifts = pd.read_csv(r"RAW/nhl_shifts_20212022.csv")

# Cleaning play-by-play data

The shift data is already quite clean but the play-by-play data needs a lot of work before we are ready to start aggregating it by player

In [30]:
df_pbp.head(5)

,Unnamed: 0,Game_Id,Date,Period,Event,Description,Time_Elapsed,Seconds_Elapsed,Strength,Ev_Zone,...,Away_Score,Home_Score,Away_Goalie,Away_Goalie_Id,Home_Goalie,Home_Goalie_Id,xC,yC,Home_Coach,Away_Coach
0,0,20001,2021-10-12,1,PSTR,Period Start- Local time: 7:43 EDT,0:00,0.0,5x5,NaN,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,NaN,NaN,Jon Cooper,Mike Sullivan
1,1,20001,2021-10-12,1,FAC,PIT won Neu. Zone - PIT #77 CARTER vs TBL #21 ...,0:00,0.0,5x5,Neu,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,0.0,0.0,Jon Cooper,Mike Sullivan
2,2,20001,2021-10-12,1,HIT,"TBL #18 PALAT HIT PIT #77 CARTER, Off. Zone",0:18,18.0,5x5,Off,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,46.0,40.0,Jon Cooper,Mike Sullivan
3,3,20001,2021-10-12,1,STOP,PUCK IN NETTING,0:38,38.0,5x5,NaN,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,NaN,NaN,Jon Cooper,Mike Sullivan
4,4,20001,2021-10-12,1,FAC,TBL won Neu. Zone - PIT #9 RODRIGUES vs TBL #7...,0:38,38.0,5x5,Neu,...,0,0,TRISTAN JARRY,8477465.0,ANDREI VASILEVSKIY,8476883.0,0.0,0.0,Jon Cooper,Mike Sullivan


In [31]:
# We have three goals that are missing a 'type' - I could just drop these but since the number is so low,
# I decided to manually fill it in by watching the game highlights to see how each goal was scored.
list_shots = ['SNAP SHOT', 'BACKHAND', 'BACKHAND']
df_pbp.iloc[[96417, 110893, 225472],6] = list_shots

In [32]:
#I want to exclude goalies from my analysis as it will skew all of my offensive and defensive features
Goalies = df_pbp['Home_Goalie'].append(df_pbp['Away_Goalie']).unique()
df_pbp = df_pbp[~df_pbp['p1_ID'].isin(Goalies) & ~df_pbp['p2_ID'].isin(Goalies)]

In [33]:
#Drop all unnecessary columns
columns = ['Game_Id','Date','Period','Event', 'Description', 'Strength', 'Type',
      'Away_Team', 'Home_Team', 'p1_name', 'p1_ID',
       'p2_name', 'p2_ID', 'p3_name', 'p3_ID']

df_pbp = df_pbp[columns]

In [34]:
#Include the type of shot used in the shot on goal or goal

df_pbp['Event'].replace('MISS', 'SHOT', inplace = True) # I decided to group in shots-on-goal with misses

def concat_event_type(row):
    #shot or goal type
    if row['Event'] in ['SHOT', 'GOAL']:
        return f"{row['Event']} ({row['Type']})"
    #fighting or regular penalty
    elif row['Event'] == 'PENL' and row['Type'] in ['Fighting (maj)', 'Instigator(2 min)',
                                                'Instigator - Misconduct(10 min)', 'Aggressor(10 min)'
                                               ]:
        return f"{row['Event']} (FIGHTING)"
   
    elif row['Event'] == 'PENL':
        return f"{row['Event']} (OTHER)"
    else:
        return row['Event']

# apply the function to create a new column
df_pbp['Event_New'] = df_pbp.apply(concat_event_type, axis=1)

#drop original Event and Type Column as they are no longer needed
df_pbp.drop(['Event', 'Type'], axis='columns', inplace=True)


In [35]:
#Let's now filter only for the events that we care about (goals, hit, shots, takeaways, giveaways, etc.)
relevant_events = ['HIT','SHOT (WRIST SHOT)', 'TAKE', 'BLOCK',
       'SHOT (SLAP SHOT)', 'SHOT (BACKHAND)', 'SHOT (TIP-IN)',
       'SHOT (SNAP SHOT)','GOAL (SNAP SHOT)',
       'GOAL (WRIST SHOT)', 'SHOT (WRAP-AROUND)', 'SHOT (DEFLECTED)',
       'PENL (OTHER)', 'GOAL (TIP-IN)', 'GOAL (BACKHAND)', 'GOAL (DEFLECTED)', 'GOAL (SLAP SHOT)',
       'PENL (FIGHTING)', 'GOAL (WRAP-AROUND)']

df_pbp = df_pbp.loc[df_pbp['Event_New'].isin(relevant_events)]


# Cleaning Shift Data

In [36]:
#The only thing I have to do this is, similar to what I did above, exclude goalies from this dataset
df_shifts = df_shifts[~df_shifts['Player'].isin(Goalies)]

# Creating Aggregate Dataframe

In [37]:
#Aggregating assists by player
def total_assists_by_player():
    df_filtered = df_pbp.loc[(df_pbp['Event_New'].str.startswith('GOAL')) & (df_pbp['p2_ID'].notnull() | df_pbp['p3_ID'].notnull())]
    assists = pd.concat([df_filtered['p2_ID'], df_filtered['p3_ID']])
    player_total_assists = assists.value_counts()
    df_assists= pd.DataFrame({'Player_Id': player_total_assists.index, 'Assists': player_total_assists.values})
    aggregate = pd.merge(df_shifts.groupby('Player_Id').first()['Player'].reset_index(), 
                         df_assists, on = 'Player_Id', how = 'left')
    return aggregate

aggregate = total_assists_by_player()

In [38]:
#Calculating total number of shifts by player
shifts = df_shifts.groupby('Player_Id')['Player_Id'].count()
player_total_shifts = pd.DataFrame({'Player_Id': shifts.index, 'Shifts': shifts.values})
aggregate_2 = pd.merge(aggregate, player_total_shifts, on ='Player_Id', how = 'left')


In [41]:
#Calculating season totals, for every event, for every player
def create_event_count():
    # Create a new dataframe with index as the union of ids in p1_ID and p2_ID
    id_union = df_pbp['p1_ID'].append(df_pbp['p2_ID']).unique()
    event_counts = pd.DataFrame(index=id_union)

    # Group the dataframe by event and p1_name, then count the occurrences of each id
    counts = df_pbp.groupby(['Event_New', 'p1_ID'])['p1_ID'].count().unstack(fill_value=0)

    # Transpose the counts dataframe to have events as columns and players as index
    event_counts = counts.transpose()
    event_counts.reset_index(inplace = True)
    
    #merging with existing aggregate dataframe and returning result
    aggregate_final = pd.merge(aggregate_2, event_counts, 
                               left_on = 'Player_Id', right_on = 'p1_ID', how = 'left').drop('p1_ID', axis = 'columns')
    
    return aggregate_final

aggregate_final = create_event_count()    


Event_New      p1_ID  BLOCK  GOAL (BACKHAND)  GOAL (DEFLECTED)  \
0          8465009.0     90                0                 0   
1          8466138.0     10                0                 0   
2          8469455.0     24                0                 1   
3          8469608.0      0                0                 0   
4          8470281.0    143                0                 0   
...              ...    ...              ...               ...   
1017       8482815.0      0                0                 0   
1018       8482834.0      5                0                 0   
1019       8483397.0      2                1                 0   
1020       8483565.0     12                0                 0   
1021       8483570.0      1                1                 0   

Event_New  GOAL (SLAP SHOT)  GOAL (SNAP SHOT)  GOAL (TIP-IN)  \
0                         2                 0              0   
1                         0                 0              3   
2              

In [40]:
aggregate_final

,Player_Id,Player,Assists,Shifts,BLOCK,GOAL (BACKHAND),GOAL (DEFLECTED),GOAL (SLAP SHOT),GOAL (SNAP SHOT),GOAL (TIP-IN),GOAL (WRAP-AROUND),GOAL (WRIST SHOT),HIT,PENL (FIGHTING),PENL (OTHER)
0,8465009,ZDENO CHARA,12.0,1730,90.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,125.0,6.0,20.0
1,8466138,JOE THORNTON,5.0,530,10.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,20.0,0.0,5.0
2,8469455,JASON SPEZZA,14.0,1208,24.0,0.0,1.0,3.0,1.0,0.0,0.0,7.0,39.0,0.0,14.0
3,8470281,DUNCAN KEITH,24.0,1920,143.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,30.0,0.0,13.0
4,8470600,RYAN SUTER,28.0,2387,99.0,1.0,1.0,1.0,1.0,0.0,0.0,3.0,51.0,0.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,8482815,KIRILL SEMYONOV,NaN,45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
1002,8482834,JESPER FRODEN,NaN,116,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,1.0
1003,8483397,MARC MCLAUGHLIN,NaN,166,2.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,15.0,0.0,2.0
1004,8483565,NICK BLANKENBURG,2.0,155,12.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,19.0,0.0,2.0
